In [1]:
import tensorflow as tf

# Get the GPU device name.
device_name = tf.test.gpu_device_name()

# The device name should look like the following:
if device_name == '/device:GPU:0':
    print('Found GPU at: {}'.format(device_name))
else:
    raise SystemError('GPU device not found')

Found GPU at: /device:GPU:0


In [2]:
from google.colab import drive
drive.mount('/contents/')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /contents/


In [3]:
import numpy as np
import pandas as pd

In [4]:
yelp_reviews=pd.read_csv("/contents/My Drive/review_data_mexican.csv",usecols=["useful","text", "cuisine"])

In [5]:
yelp_reviews.head()

,cuisine,useful,text
0,Mexican,0,Great basics. Beans are made right. Tortillas ...
1,Mexican,0,Automatic star awarded for having a salsa bar ...
2,Mexican,0,Got the #7 Enchilada and Burrito great food an...
3,Mexican,0,good food for the price. I like their new loca...
4,Mexican,1,One of the better breakfast burritos I have ha...


In [6]:
yelp_reviews["labels"]= yelp_reviews["useful"].apply(lambda x: 1 if x >= 1  else 0)

In [7]:
texts = yelp_reviews.loc[:1000, "text"]
labels = yelp_reviews.loc[:1000, "labels"]

In [8]:
texts[0:6]

0    Great basics. Beans are made right. Tortillas ...
1    Automatic star awarded for having a salsa bar ...
2    Got the #7 Enchilada and Burrito great food an...
3    good food for the price. I like their new loca...
4    One of the better breakfast burritos I have ha...
5    Amado's Bros is a new favorite Friday breakfas...
Name: text, dtype: object

In [10]:
labels[0:6]

0    0
1    0
2    0
3    0
4    1
5    0
Name: labels, dtype: int64

In [11]:
max_length = 50 #512
batch_size = 6


In [12]:
!pip install transformers

     |████████████████████████████████| 778kB 7.2MB/s 
     |████████████████████████████████| 3.0MB 38.7MB/s 
     |████████████████████████████████| 890kB 36.0MB/s 
     |████████████████████████████████| 1.1MB 33.5MB/s 
  Created wheel for sacremoses: filename=sacremoses-0.0.43-cp36-none-any.whl size=893260 sha256=1d4446135616d1a6fe4468f4b952a9d36e33766211646e6acc9a09052c36e029
  Stored in directory: /root/.cache/pip/wheels/29/3c/fd/7ce5c3f0666dab31a50123635e6fb5e19ceb42ce38d4e58f45
Successfully built sacremoses


In [13]:
from transformers import BertTokenizer

tokenizer = BertTokenizer.from_pretrained('bert-base-uncased', do_lower_case=True)

In [55]:
import tensorflow as tf

In [14]:
def convert_example_to_feature(review):
  
  # combine step for tokenization, WordPiece vector mapping, adding special tokens as well as truncating reviews longer than the max length
  
  return tokenizer.encode_plus(review, 
                add_special_tokens = True, # add [CLS], [SEP]
                max_length = max_length, # max length of the text that can go to BERT
                pad_to_max_length = True, # add [PAD] tokens
                return_attention_mask = True, # add attention mask to not focus on pad tokens
                truncation=True
              )

In [15]:
convert_example_to_feature("this is just a test")

{'input_ids': [101, 2023, 2003, 2074, 1037, 3231, 102, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], 'token_type_ids': [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]}

Method 1

In [68]:
# map to the expected input to TFBertForSequenceClassification, see here 
def map_example_to_dict(input_ids, attention_masks, token_type_ids, label):
  return {
      "input_ids": input_ids,
      "token_type_ids": token_type_ids,
      "attention_mask": attention_masks,
  }, label

def encode_examples(texts, labels):

  # prepare list, so that we can build up final TensorFlow dataset from slices.
  input_ids_list = []
  token_type_ids_list = []
  attention_mask_list = []
  label_list = []
    
  for review, label in zip(texts, labels):

    bert_input = convert_example_to_feature(review)
  
    input_ids_list.append(bert_input['input_ids'])
    token_type_ids_list.append(bert_input['token_type_ids'])
    attention_mask_list.append(bert_input['attention_mask'])

    padding = np.zeros(max_length - 1)
    item = []
    item.append(label)
    item.extend(padding)

    label_list.append(item)

  return tf.data.Dataset.from_tensor_slices((input_ids_list, attention_mask_list, token_type_ids_list, label_list)).map(map_example_to_dict)


In [69]:
x_train = encode_examples(texts, labels)

In [70]:
for element in x_train:
  print(element)

Streaming output truncated to the last 5000 lines.
array([  101,  2572,  1045,  1999,  3290,  1029,  2572,  1045,  1999,
        6708,  1029,  1045,  1005,  1049,  2061,  5457,   999,  2008,
        1005,  1055,  2054,  2474, 14124,  3972,  3814,  5428,  2097,
        2079,  2000,  2017,  1012,  1996,  2617,  2017,  3357,  2083,
        1996,  4303,  1010,  2017,  1005,  2222,  2514,  2066,  2017,
        1005,  2128,  1999,  3290,   102], dtype=int32)>, 'token_type_ids': <tf.Tensor: shape=(50,), dtype=int32, numpy=
array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0], dtype=int32)>, 'attention_mask': <tf.Tensor: shape=(50,), dtype=int32, numpy=
array([1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1], dtype=int32)>}, <tf.Tensor: shape=(50,), dtype=float32, 

Method 2

In [56]:
def encode_examples(texts):

  # prepare list, so that we can build up final TensorFlow dataset from slices.
    input_ids_list = []
    token_type_ids_list = []
    attention_mask_list = []
    label_list = []
    
    for review in texts:

        bert_input = convert_example_to_feature(review)

        input_ids_list.append(bert_input['input_ids'])
        token_type_ids_list.append(bert_input['token_type_ids'])
        attention_mask_list.append(bert_input['attention_mask'])
        
        bert_inputs = np.array([input_ids_list, attention_mask_list, token_type_ids_list])
        

    return bert_inputs

In [33]:
texts.shape[0]

1001

In [46]:
y_train = []
item = []
padding = np.zeros(max_length - 1)

for i in labels:
  item.append(i)
  item.extend(padding)
  y_train.append(item)
  item = []

y_train[: 1]

[[0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0]]

In [49]:
y_train = np.array(y_train)
y_train.shape

(1001, 50)

In [52]:
y_train_1 = np.array(labels)
y_train_1.shape

(1001,)

In [30]:
x_train.shape

(3, 1001, 50)

In [50]:
from transformers import TFBertForSequenceClassification
import tensorflow as tf

# recommended learning rate for Adam 5e-5, 3e-5, 2e-5
learning_rate = 2e-5

# we will do just 1 epoch for illustration, though multiple epochs might be better as long as we will not overfit the model
number_of_epochs = 1


# model initialization
model = TFBertForSequenceClassification.from_pretrained('bert-base-uncased')

# choosing Adam optimizer
optimizer = tf.keras.optimizers.Adam(learning_rate=learning_rate, epsilon=1e-08)

# we do not have one-hot vectors, we can use sparce categorical cross entropy and accuracy
loss = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)
metric = tf.keras.metrics.SparseCategoricalAccuracy('accuracy')

model.compile(optimizer=optimizer, loss=loss, metrics=[metric])

Some weights of the model checkpoint at bert-base-uncased were not used when initializing TFBertForSequenceClassification: ['nsp___cls', 'mlm___cls']
- This IS expected if you are initializing TFBertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPretraining model).
- This IS NOT expected if you are initializing TFBertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of TFBertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['dropout_75', 'classifier']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [71]:
model.fit(x_train, epochs=number_of_epochs)

ValueError: ignored